In [1]:
from skmap.io import RasterData
from skmap.io import process
import os

from pathlib import Path
from minio import Minio
from eumap.misc import find_files, nan_percentile
import numpy as np
from minio.commonconfig import REPLACE, CopySource
import time as tm
import multiprocess as mp

tile_ids = ['014E_59N','006W_37N','008E_45N','005E_52N','024E_43N']
year_list = list(range(2000,2022,1))
bands = ['blue','green','red','NIR','SWIR1','SWIR2','thermal']


In [3]:
def worker_move(src_f):
    # start = tm.time()
    # configuration
    s3_config = {
    'access_key': 'iwum9G1fEQ920lYV4ol9',
    'secret_access_key': 'GMBME3Wsm8S7mBXw3U4CNWurkzWMqGZ0n2rXHggS0',
    'host': '192.168.1.30:8333',
    'bucket': 'tmp-bare-soil'}
    
    fn = src_f.split('/')[-1]
    client = Minio(s3_config['host'], s3_config['access_key'], s3_config['secret_access_key'], secure=False)
    s3_path = src_f[26::]

    
    client.fput_object(s3_config['bucket'], s3_path, src_f)
    # end = tm.time()
    
    # print(f'{fn} finished in {round((end-start)/60)} mins')
    return None

In [4]:
files = find_files('/mnt/tupi/Xuemeng_testing/original','landsat_*.tif')
files = [str(i) for i in files]

In [8]:
with mp.Pool(processes=50) as pool:
    result = pool.map(worker_move, files)

In [14]:
def worker_aggregate(year,band,tile,period):
    
    ff = f'/mnt/tupi/Xuemeng_testing/landsat_{period}/{band}_landsat.{tile}_p50_{str(year+1)}001.tif'

    # if os.path.exists(ff):
    #     print(f'{ff} exists!')
    #     return None
    # else:
    #     print(f'{ff} generating')
                    
    
    ks = f'http://192.168.1.30:8333/tmp-bare-soil/original/{band}/'
    ke = f'/landsat_{tile}.tif'
    rdata = RasterData({
        f'{band}': ks + '{dt}' + ke
    }, verbose=True).timespan(f'{str(year)}/001', f'{str(year+1)}/001', date_step=16, date_unit='days', date_format='%Y/%j', date_style='start_date') \
    .read()
    
    base_raster_path = rdata._base_raster()
    
    outf = '{gr}_landsat.' + f'{tile}' + '_{op}_{dt}'
    
    if period == 'monthly':
        processor = process.TimeEnum.MONTHLY_15P
    elif period == 'bimonthly':
        processor = process.TimeEnum.BIMONTHLY_15P
        
    rdata = rdata.run(
        process.TimeAggregate(
            time=[processor], operations = ['p50'], verbose=True
        ), outname=outf, drop_input=True
    ).rename(groups= {f'{band}..p50': f'{band}'}
            )
    
    rdata.info['name'] = rdata.info['name'].apply(lambda x: x.replace('/',''))
    rdata.info['input_path'] = base_raster_path
    
    rdata.to_dir(f'landsat_{period}')
    return None

In [15]:
worker_aggregate(2004,'thermal','014E_59N','monthly')

[16:36:04] RasterData with 23 rasters and 1 groups
[16:36:04] Reading 23 raster file(s) using 4 workers
[16:36:13] Read array shape: (4004, 4004, 23)
[16:36:16] Running TimeAggregate on (4004, 4004, 23) for thermal group
[16:36:20] Computing 13 time aggregates from 2004 to 2005
[16:36:27] Dropping data and info for thermal group
[16:36:27] Execution time for TimeAggregate: 14.53 segs
[16:36:27] Saving rasters in landsat_monthly
[16:36:29] Saving 13 raster files using 4 workers


In [2]:
tem = find_files('/mnt/tupi/Xuemeng_testing/landsat_monthly/','*landsat*.tif')
tem = [str(i) for i in tem]
len(tem)-22*5*12*7

34

In [7]:
tem[0].split('_')[-1]

'2000001.tif'

In [4]:
date = []
for i in tem:
    date.append(i.split('_')[-1])
    
date = list(set(date))

In [5]:
temf = []
for period in ['monthly']:
    for ts in tile_ids:
        for bs in bands:
            for ys in date:
                temf.append(f'/mnt/tupi/Xuemeng_testing/landsat_{period}/{bs}_landsat.{ts}_p50_{ys}')

In [11]:
# rm = ['/mnt/tupi/Xuemeng_testing/landsat_monthly/SWIR1_landsat.006W_37N_p50_2018001.tif',
#       '/mnt/tupi/Xuemeng_testing/landsat_monthly/blue_landsat.008E_45N_p50_2017001.tif',
#       '/mnt/tupi/Xuemeng_testing/landsat_monthly/NIR_landsat.008E_45N_p50_2016001.tif',
#       '/mnt/tupi/Xuemeng_testing/landsat_monthly/thermal_landsat.008E_45N_p50_2015001.tif',
#       '/mnt/tupi/Xuemeng_testing/landsat_monthly/red_landsat.005E_52N_p50_2014001.tif',
#       '/mnt/tupi/Xuemeng_testing/landsat_monthly/SWIR2_landsat.005E_52N_p50_2013001.tif',
#       '/mnt/tupi/Xuemeng_testing/landsat_monthly/green_landsat.024E_43N_p50_2012001.tif',
#       '/mnt/tupi/Xuemeng_testing/landsat_monthly/SWIR1_landsat.024E_43N_p50_2011001.tif']
   
# for i in rm:
#     os.remove(i)
      
      
      

In [6]:
omit = []
for i in temf:
    if i not in tem:
        omit.append(i)
        print(i)

/mnt/tupi/Xuemeng_testing/landsat_monthly/thermal_landsat.014E_59N_p50_2004182.tif


In [31]:
len(tem)

9187

In [32]:
# inputs =[]
# for period in ['monthly','bimonthly']:
#     for ts in tile_ids:
#         for bs in bands:
#             for ys in year_list:
#                 inputs.append((ys,bs,ts,period))
                
# with mp.Pool(processes=6) as pool:
#     result = pool.starmap(worker_aggregate, inputs)

In [6]:
for period in ['monthly','bimonthly']:
    for ts in tile_ids:
        for bs in bands:
            for ys in year_list:
                # inputs.append((ys,bs,ts,period))
                ff = f'/mnt/tupi/Xuemeng_testing/landsat_{period}/{bs}_landsat.{ts}_p50_{str(ys+1)}001.tif'
                fn = ff.split('/')[-1]
                
                if os.path.exists(ff):
                    continue
                else:
                    worker_aggregate(ys,bs,ts,period)
                    print(f'{fn} generated')
                    print()

/mnt/tupi/Xuemeng_testing/landsat_monthly/SWIR1_landsat.006W_37N_p50_2018001.tif generating
[19:36:15] RasterData with 23 rasters and 1 groups
[19:36:15] Reading 23 raster file(s) using 4 workers
[19:36:26] Read array shape: (4004, 4004, 23)
[19:36:29] Running TimeAggregate on (4004, 4004, 23) for SWIR1 group
[19:36:33] Computing 13 time aggregates from 2017 to 2018
[19:36:40] Dropping data and info for SWIR1 group
[19:36:40] Execution time for TimeAggregate: 14.46 segs
[19:36:40] Saving rasters in landsat_monthly
[19:36:42] Saving 13 raster files using 4 workers
SWIR1_landsat.006W_37N_p50_2018001.tif generated

/mnt/tupi/Xuemeng_testing/landsat_monthly/blue_landsat.008E_45N_p50_2017001.tif generating
[19:36:45] RasterData with 23 rasters and 1 groups
[19:36:45] Reading 23 raster file(s) using 4 workers
[19:36:55] Read array shape: (4004, 4004, 23)
[19:36:58] Running TimeAggregate on (4004, 4004, 23) for blue group
[19:37:02] Computing 13 time aggregates from 2016 to 2017
[19:37:09] Dr

In [5]:
files = find_files('/mnt/tupi/Xuemeng_testing/monthly_gapfilled','*_landsat*.tif')
ff = []
for i in files:
    # ii = str(i).split('_')[-1]
    # if ii == '2022001.tif':
    os.remove(str(i))

In [4]:
def worker_gapfill(ts,bs,period):
    
    # bands = ['blue','green','red','NIR','SWIR1','SWIR2','thermal']
    # fgroups = [f'/mnt/tupi/Xuemeng_testing/landsat_monthly/{bs}_landsat.{ts}_p50_'+'{dt}.tif' for bs in bands]
    # group = dict(zip(bands, fgroups))
    group = {f'{bs}': f'/mnt/tupi/Xuemeng_testing/landsat_monthly/{bs}_landsat.{ts}_p50_'+'{dt}.tif'}
    if period == 'monthly':
        ks = 1
    else:
        ks = 2
        
    rdata = RasterData(group,verbose=True) \
    .timespan('2000001', '2021353', date_step=ks, date_unit='months', date_format='%Y%j', date_style='start_date') \
    .read()
    
    outn = 'landsat_{gr}_'+f'{ts}' + '_{dt}'
    base_raster_path = rdata._base_raster()
    rdata = rdata.run(process.SeasConvFill(season_size=4), outname=outn, drop_input=True).rename(groups= {'{gr}..p50': '{gr}'})
    rdata.info['input_path'] = base_raster_path
    rdata.to_dir(f'{period}_gapfilled')
    
    return None

In [3]:
for period in ['monthly','bimonthly']:
    for bs in bands:
        for ts in tile_ids:
            ff = f'/mnt/tupi/Xuemeng_testing/{period}_gapfilled/landsat_{bs}_{ts[0:4]}.{ts[5::]}_2021335.tif'
            print(ff)
            fn = ff.split('/')[-1]
            if os.path.exists(ff):
                print(f'it exists!')
                print()
            else:
                worker_gapfill(ts,bs,period)

/mnt/tupi/Xuemeng_testing/monthly_gapfilled/landsat_blue_014E.59N_2021335.tif
[13:14:14] RasterData with 264 rasters and 1 groups
[13:14:14] Reading 264 raster file(s) using 4 workers


KeyboardInterrupt: 

In [42]:
from skmap.data import toy
from skmap.io import process

# Loading NDVI quarterly time-series with gaps
toy.ndvi_rdata(gappy=True 
    # Gapfilling time-series by seasonal convolution 
    ).run(process.SeasConvFill(season_size=4), drop_input=True
    # Smoothing time-series by Whittaker
    ).run(process.WhittakerSmooth(), group='ndvi.seasconv', drop_input=True
    # Setting smoothed time-series as main input
    ).rename(groups={'ndvi.seasconv.whittaker': 'ndvi'}
    # Running yearly aggregation by std. and percentile 50th
    ).run(process.TimeAggregate(time=[process.TimeEnum.YEARLY], operations = ['p50', 'std']), group=['ndvi']
    # Running trend analysis using per-pixel linear regression  
    ).run(process.TrendAnalysis(season_size=4), group='ndvi'
    # Ploting all raster data
    ).plot(v_minmax=[0,100])

In [10]:
os.path.exists('/mnt/tupi/Xuemeng_testing/landsat_test/'+ff)

True

In [10]:
mnt/tupi/Xuemeng_testing/landsat_test/blue_landsat.008E_45N_p50_2000001.tif

[08:40:59] RasterData with 23 rasters and 1 groups
[08:40:59] Reading 23 raster file(s) using 4 workers
[08:41:08] Read array shape: (4004, 4004, 23)
[08:41:11] Running TimeAggregate on (4004, 4004, 23) for blue group
[08:41:16] Computing 13 time aggregates from 2000 to 2001
[08:41:22] Dropping data and info for blue group
[08:41:23] Execution time for TimeAggregate: 14.80 segs
[08:41:23] Saving rasters in landsat_test
[08:41:25] Saving 13 raster files using 4 workers
